# Calculate Z-scores for players

This takes the list of players that we're trying to calculate scores for, and creates z-scores for players with an arbitrarily chosen minimum innings (for pitchers) or plate appearances (for hitters).

Notes for this year's sheet:

- Z-score avg/stdev calculated using minimum IP (70) or PA (35) stats for batting and pitching.
- Reliever value was artificially decremented as usual since they're normally over-inflated value-wise. Multiplied by (project innings) / (90th pctile of projected innings), capped at 1.0.

In [1]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
import psycopg2
import seaborn as sb
import requests

# arbitrarily high max view columns
pd.options.display.max_columns = 150
pd.options.display.max_rows = 200

/Users/zach/Library/Python/3.7/lib/python/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## SqlAlchemy Connection Information

These are used to get and return a connection to the postgres DB so that we can query for player stats and write them back to the table.

In [2]:
# connection information for the database
POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_IP = "192.168.0.118"
POSTGRES_PORT = 5432
POSTGRES_DB = "postgres"

def get_sqlalchemy_engine():
    """
    Create and return a SQLAlchemy engine for inserting into postgres.
    """
    # ## Write Information Back to Database
    #
    return sqlalchemy.create_engine(
        "postgres://{user}:{password}@{host}:{port}/{db}".format(
            user=POSTGRES_USER,
            password=POSTGRES_PASSWORD,
            host=POSTGRES_IP,
            port=POSTGRES_PORT,
            db=POSTGRES_DB,
        )
    )

In [3]:
# create a connection and read in pitchers data
engine = get_sqlalchemy_engine()
conn = engine.connect()

## Calculate Z-Scores

For each of the players, we calculate a z-score based on how they compare to the rest of the league. There are several columns that we want to do this for.

- For each z-score, there is an arbitrary minimum set to inflate the league average away from non-full-time players.
- After each is totaled, the z-scores are totaled and then z-scored again to make a single number. This is done so that we can compare pitchers and batters into the same chart.

In [4]:
# read in information from postgres
dfp = pd.read_sql("select * from fantasy.pitchers_projections_depth_charts_ros where ip > 0.0", conn)
dfb = pd.read_sql("select *from fantasy.batters_projections_depth_charts_ros", conn)
dfpa = pd.read_sql("select * from fantasy.pitchers_actuals", conn)
dfba = pd.read_sql("select * from fantasy.batters_actuals", conn)
# dfpa.fillna(0, inplace=True)
# dfba.fillna(0, inplace=True)
print()

In [20]:
# create any calculated columns necessary
dfb['k_pct'] = (dfb['so'] / dfb['ab']).round(2)
dfb['rc'] = (dfb['ab'] * dfb['obp'] * dfb['slg']).round(2)
dfba['k_pct'] = (dfb['so'] / dfb['ab']).round(2)
dfba['ab'] = (dfba['pa'] * dfba['bb_pct'].apply(lambda x: 1 - float(x.strip('%')) / 100)).astype(int)
dfba['rc'] = (dfba['ab'] * dfba['obp'] * dfba['slg']).round(2)
dfba['ops'] = dfba['slg'] + dfba['obp']

# quality starts metric from here:
# https://yourfantasyjoe.blogspot.com/2010/02/projecting-quality-starts.html
# xQS = GS * (.4650115 - (ERA * .0872381) + ((IP/GS) * .0746775))
dfp['qs'] = (dfp['gs'] * (0.4650115 - (dfp['era'] * 0.0872381) + ((dfp['ip']/dfp['gs']) * .0746775))).round(2)
dfp.fillna(value={'qs': 0.0}, inplace=True)
print()

In [27]:
# thresholds for batters and pitchers
MINIMUM_INNINGS_PITCHED = 100
MINIMUM_INNINGS_PITCHED_ACTUALS = 4.0
MINIMUM_PLATE_APPEARANCES = 180
MINIMUM_PLATE_APPEARANCES_ACTUALS = 20
min_pa = dfb['pa'] > MINIMUM_PLATE_APPEARANCES
min_pa_act = dfba['pa'] > MINIMUM_PLATE_APPEARANCES_ACTUALS
min_ip = dfp['ip'] > MINIMUM_INNINGS_PITCHED
min_ip_act = dfpa['ip'] > MINIMUM_INNINGS_PITCHED_ACTUALS

# dict of columns that we want to calculate z-scores for
# +1 means more is better, -1 means lower is better
dfb_score_cols = {
    "pa": {"dir": 1, "weight": 1.3}, 
    "k_pct": {"dir": -1, "weight": 1.0},
    "hr": {"dir": 1, "weight": 0.9},
    "rc": {"dir": 1, "weight": 1.4},
    "woba": {"dir": 1, "weight": 1.3},
    "slg": {"dir": 1, "weight": 1.0},
    "adp": {"dir": -1, "weight": 0.3}
}
dfp_score_cols = {
    "ip": {"dir": 1, "weight": 1.3},
    "era": {"dir": -1, "weight": 1.0},
    "hr": {"dir": -1, "weight": 0.9},
    "so": {"dir": 1, "weight": 1.0},
    "whip": {"dir": -1, "weight": 1.5},
    "k-9": {"dir": 1, "weight": 1.3},
    "qs": {"dir": 1, "weight": 1.0},
    "gs": {"dir": 1, "weight": 0.35}
}
dfba_score_cols = {
    "pa": {"dir": 1, "weight": 2.15},
    "k_pct": {"dir": -1, "weight": 0.4},
    "hr": {"dir": 1, "weight": 1.25},
    "rc": {"dir": 1, "weight": 1.12},
    "ops": {"dir": 1, "weight": 1.015},
    "xwoba": {"dir": 1, "weight": 1.15},
    "iso": {"dir": 1, "weight": 1.1}
}
dfpa_score_cols = {
    "ip": {"dir": 1, "weight": 1.0},
    "hr_per9": {"dir": -1, "weight": 0.9},
    "k_per9": {"dir": 1, "weight": 1.0},
    "whip": {"dir": -1, "weight": 1.5},
    "xfip": {"dir": -1, "weight": 1.0},
    "siera": {"dir": -1, "weight": 1.15},
}
dfb_weight = 1.0
dfba_weight = 0.85
dfp_weight = 1.0
dfpa_weight = 0.82

# get all of the score columns for maximum z-scoring
dfb_score_col_names = [x + "_score" for x in dfb_score_cols.keys()]
dfba_score_col_names = [x + "_score" for x in dfba_score_cols.keys()]
dfp_score_col_names = [x + "_score" for x in dfp_score_cols.keys()]
dfpa_score_col_names = [x + "_score" for x in dfpa_score_cols.keys()]

# filter out unqualifieds
dfba = dfba[min_pa_act]
dfpa = dfpa[min_ip_act]

In [26]:
dfpa.head()

,index,idx,name,team,k_per9,bb_per9,k_perbb,hr_per9,k_pct,bb_pct,k_bb_pct,so,avg,whip,babip,lob_pct,era_,fip_,xfip_,era,fip,e_f,xfip,siera,ip,ip_score,hr_per9_score,k_per9_score,whip_score
4,4,5,Craig Kimbrel,CHC,14.63,5.63,2.6,0.0,43.3%,16.7%,26.7%,13,0.040,0.75,0.083,100.0%,0,40,62,0.0,1.71,-1.71,2.58,2.89,8.0,-0.464,0.953,1.591,1.770
6,6,7,Ryan Pressly,HOU,9.00,1.50,6.0,0.0,25.0%,4.2%,20.8%,6,0.261,1.17,0.353,100.0%,0,42,55,0.0,1.59,-1.59,2.16,1.91,6.0,-0.797,0.953,-0.176,0.393
7,7,8,Brad Hand,WSN,7.50,3.00,2.5,0.0,20.8%,8.3%,12.5%,5,0.150,0.83,0.200,100.0%,0,82,120,0.0,3.42,-3.42,4.86,3.93,6.0,-0.797,0.953,-0.647,1.508
8,8,9,Aroldis Chapman,NYY,23.40,3.60,6.5,0.0,68.4%,10.5%,57.9%,13,0.118,0.80,0.500,100.0%,0,-18,-2,0.0,-0.91,0.91,-0.22,0.49,5.0,-0.963,0.953,4.344,1.606
11,11,12,Luke Jackson,ATL,8.53,8.53,1.0,0.0,21.4%,21.4%,0.0%,6,0.227,1.74,0.313,100.0%,0,100,134,0.0,4.03,-4.03,5.40,5.89,6.1,-0.780,0.953,-0.324,-1.476


In [28]:
# only mean and stdev are filtered by minimums, so that way the player's actual
# projections are still counted normally.

# batter scores
for col in dfb_score_cols.keys():
    col_score = col + "_score"
    dfb[col_score] = (
        (dfb[col] - dfb[col][min_pa].mean()) / dfb[col][min_pa].std(ddof=0)
        * dfb_score_cols[col]["dir"]
        * dfb_score_cols[col]["weight"]
    ).round(3)

# batter actuals
for col in dfba_score_cols.keys():
    col_score = col + "_score"
    dfba[col_score] = (
        (dfba[col] - dfba[col][min_pa_act].mean()) / dfba[col][min_pa_act].std(ddof=0)
        * dfba_score_cols[col]["dir"]
        * dfba_score_cols[col]["weight"]
    ).round(3)
    
# pitcher scores
for col in dfp_score_cols.keys():
    col_score = col + "_score"
    dfp[col_score] = (
        (dfp[col] - dfp[col][min_ip].mean()) / dfp[col][min_ip].std(ddof=0)
        * dfp_score_cols[col]["dir"]
        * dfp_score_cols[col]["weight"]
    ).round(3)

# pitcher actuals
for col in dfpa_score_cols.keys():
    col_score = col + "_score"
    dfpa[col_score] = (
        (dfpa[col] - dfpa[col][min_ip_act].mean()) / dfpa[col][min_ip_act].std(ddof=0)
        * dfpa_score_cols[col]["dir"]
        * dfpa_score_cols[col]["weight"]
    ).round(3)

In [29]:
# sum all of the values into 'total_score'
dfb['total_score'] = dfb[dfb_score_col_names].sum(axis=1)
dfb['total_z_score'] = (
    dfb['total_score'] - dfb['total_score'][min_pa].mean()) / dfb['total_score'][min_pa].std(ddof=0)
dfb['total_z_score_rank'] = dfb['total_z_score'].rank(ascending=False)

dfba['total_score'] = dfba[dfba_score_col_names].sum(axis=1)
dfba['total_z_score'] = (
    dfba['total_score'] - dfba['total_score'][min_pa_act].mean()) / dfba['total_score'][min_pa_act].std(ddof=0)
dfba['total_z_score_rank'] = dfba['total_z_score'].rank(ascending=False)

dfp['total_score'] = dfp[dfp_score_col_names].sum(axis=1)
dfp['total_z_score'] = (
    dfp['total_score'] - dfp['total_score'][min_ip].mean()) / dfp['total_score'][min_ip].std(ddof=0)
dfp['total_z_score_rank'] = dfp['total_z_score'].rank(ascending=False)

dfpa['total_score'] = dfpa[dfpa_score_col_names].sum(axis=1)
dfpa['total_z_score'] = (
    dfpa['total_score'] - dfpa['total_score'][min_ip_act].mean()) / dfpa['total_score'][min_ip_act].std(ddof=0)
dfpa['total_z_score_rank'] = dfpa['total_z_score'].rank(ascending=False)

# sort by score descending
dfb.sort_values(by='total_z_score_rank', inplace=True)
dfba.sort_values(by='total_z_score_rank', inplace=True)
dfp.sort_values(by='total_z_score_rank', inplace=True)
dfpa.sort_values(by='total_z_score', ascending=False, inplace=True)


### Inflate League Mean

For the majority of the z-scores that we're taking, the actual values of them are affected by the long-tail of players who don't get much playing time. To offset this, the mean is artifically placed at halfway through our number of drafted players to account for the fact that will be replacement level for our league.

In [30]:
# decrement scores by the mean of all drafted positions
NUM_TEAMS = 8
NUM_BATTERS = 15
NUM_PITCHERS = 10

MIDDLE_BATTER_INDEX = int((NUM_TEAMS * NUM_BATTERS) / 2)
MIDDLE_PITCHER_INDEX = int((NUM_TEAMS * NUM_PITCHERS) / 2)

middle_batter_score = dfb[dfb['total_z_score_rank'] == MIDDLE_BATTER_INDEX]['total_z_score']
dfb['total_z_score'] = dfb['total_z_score'] - float(middle_batter_score)
dfb.reset_index(drop=True)

middle_pitcher_score = dfp[dfp['total_z_score_rank'] == MIDDLE_PITCHER_INDEX]['total_z_score']
dfp['total_z_score'] = dfp['total_z_score'] - float(middle_pitcher_score)
dfp.reset_index(drop=True)
print()

## Top Batter Results

In [31]:
dfb.head(150)

,index,name,team,g,pa,ab,h,2b,3b,hr,r,rbi,bb,so,hbp,sb,cs,avg,obp,slg,ops,woba,fld,bsr,war,adp,k_pct,rc,pa_score,k_pct_score,hr_score,rc_score,woba_score,slg_score,adp_score,total_score,total_z_score,total_z_score_rank
0,0,Mike Trout,LAA,146,612,485,137,25,3,39,103,104,111,140,11,10,3,0.283,0.425,0.588,1.013,0.413,-2.7,1.1,6.9,6.0,0.29,121.20,1.951,-0.475,2.656,4.489,4.981,3.412,0.421,17.435,2.038534,1.0
6,6,Juan Soto,WSN,123,517,422,127,25,2,29,91,97,88,87,3,11,4,0.301,0.422,0.580,1.002,0.409,-1.6,0.0,4.4,3.7,0.21,103.29,0.893,0.761,1.525,3.282,4.775,3.247,0.424,14.907,1.602764,2.0
3,3,Ronald Acuna,ATL,133,560,473,136,26,2,35,106,87,74,136,8,25,7,0.287,0.390,0.570,0.960,0.396,2.7,2.8,4.9,1.6,0.29,105.15,1.372,-0.475,2.204,3.407,4.105,3.040,0.427,14.080,1.460208,3.0
15,15,Freddie Freeman,ATL,144,604,507,146,34,2,28,93,95,84,106,7,5,2,0.288,0.395,0.527,0.922,0.381,2.5,-0.3,3.7,13.4,0.21,105.54,1.862,0.761,1.412,3.434,3.331,2.150,0.412,13.362,1.336441,4.0
4,4,Jose Ramirez,CLE,143,599,518,141,35,3,30,89,93,71,88,5,24,7,0.273,0.364,0.526,0.890,0.369,1.5,2.6,4.8,10.9,0.17,99.18,1.806,1.379,1.638,3.005,2.713,2.129,0.415,13.085,1.288693,5.0
2,2,Alex Bregman,HOU,139,583,487,134,31,2,26,86,85,83,79,9,5,2,0.275,0.387,0.508,0.895,0.376,-1.2,-0.3,4.9,47.3,0.16,95.74,1.628,1.533,1.185,2.773,3.074,1.757,0.369,12.319,1.156652,6.0
1,1,Mookie Betts,LAD,144,604,518,144,31,2,28,103,79,74,93,6,21,5,0.278,0.372,0.507,0.880,0.368,12.0,3.6,5.2,4.1,0.18,97.70,1.862,1.224,1.412,2.905,2.661,1.736,0.424,12.224,1.140276,7.0
18,18,Bryce Harper,PHI,141,591,477,124,25,2,32,92,96,104,142,6,12,4,0.259,0.395,0.519,0.914,0.378,-1.6,-0.5,3.6,17.6,0.30,97.79,1.717,-0.629,1.864,2.911,3.177,1.984,0.407,11.431,1.003581,8.0
5,5,Corey Seager,LAD,136,572,507,145,33,2,27,87,95,55,102,5,3,1,0.286,0.360,0.516,0.876,0.365,-1.9,0.7,4.4,29.2,0.20,94.18,1.506,0.915,1.299,2.668,2.507,1.922,0.392,11.209,0.965313,9.0
27,27,Vladimir Guerrero Jr.,TOR,132,554,489,143,28,3,25,75,87,55,88,6,3,1,0.291,0.367,0.514,0.881,0.368,0.0,-1.1,3.1,45.3,0.18,92.24,1.305,1.224,1.072,2.537,2.661,1.881,0.372,11.052,0.938250,10.0


## Top Pitcher Results

In [32]:
## Top Pitchers
dfp.head(100)

,index,name,team,w,l,sv,hld,era,gs,g,ip,h,er,hr,so,bb,whip,k-9,bb-9,fip,war,adp,qs,ip_score,era_score,hr_score,so_score,whip_score,k-9_score,qs_score,gs_score,total_score,total_z_score,total_z_score_rank
0,0,Jacob deGrom,NYM,14,6,0,0,2.60,30,30,187.0,135,54,19,259,44,0.96,12.52,2.12,2.56,6.6,4.3,21.11,3.459,3.081,0.030,3.509,4.686,3.157,3.462,0.762,22.146,3.178192,1.0
1,1,Gerrit Cole,NYY,15,7,0,0,3.14,29,29,179.0,138,62,24,247,46,1.04,12.47,2.34,3.05,5.6,7.3,18.91,2.952,2.066,-1.298,3.171,3.592,3.112,2.607,0.640,16.842,2.328538,2.0
2,2,Shane Bieber,CLE,12,8,0,0,3.28,28,28,175.0,148,64,22,224,40,1.07,11.46,2.06,3.06,5.0,9.8,18.08,2.699,1.803,-0.767,2.524,3.181,2.191,2.284,0.517,14.432,1.942478,3.0
3,3,Max Scherzer,WSN,12,7,0,0,3.39,28,28,170.0,140,64,25,228,45,1.08,12.02,2.35,3.34,4.6,24.8,17.43,2.382,1.596,-1.563,2.637,3.044,2.701,2.031,0.517,13.345,1.768350,4.0
4,4,Trevor Bauer,LAD,13,6,0,0,3.36,27,27,170.0,140,63,21,211,54,1.14,11.18,2.85,3.43,4.2,15.5,17.34,2.382,1.653,-0.501,2.159,2.223,1.935,1.996,0.394,12.241,1.591499,5.0
6,6,Lucas Giolito,CHW,12,7,0,0,3.47,27,27,161.0,126,62,21,209,56,1.13,11.71,3.12,3.48,4.1,16.9,16.41,1.812,1.446,-0.501,2.102,2.360,2.419,1.635,0.394,11.667,1.499549,6.0
9,9,Yu Darvish,SDP,11,6,0,0,3.43,27,27,161.0,132,61,21,200,48,1.11,11.17,2.66,3.44,3.8,16.5,16.50,1.812,1.521,-0.501,1.849,2.634,1.926,1.670,0.394,11.305,1.441560,7.0
5,5,Aaron Nola,PHI,12,7,0,0,3.48,27,27,167.0,142,65,20,197,53,1.17,10.59,2.85,3.47,4.2,21.8,16.83,2.192,1.427,-0.236,1.765,1.812,1.397,1.798,0.394,10.549,1.320456,8.0
10,10,Tyler Glasnow,TBR,10,6,0,0,3.37,24,24,138.0,104,52,16,192,55,1.16,12.55,3.59,3.26,3.7,43.2,14.41,0.356,1.634,0.827,1.624,1.949,3.185,0.857,0.027,10.459,1.306038,9.0
7,7,Brandon Woodruff,MIL,10,7,0,0,3.45,27,27,157.0,134,60,18,189,48,1.16,10.85,2.76,3.36,4.1,32.0,16.15,1.559,1.483,0.296,1.540,1.949,1.634,1.534,0.394,10.389,1.294825,10.0


## Write to the Database

Prior to writing the excel files, write back to the database for safekeeping.

In [33]:
dfb.to_sql("batters_scores_ros", conn, schema="fantasy", if_exists="replace")
result = conn.execute("grant select on fantasy.batters_scores_ros to public")

dfba.to_sql("batters_scores_actuals", conn, schema="fantasy", if_exists="replace")
result = conn.execute("grant select on fantasy.batters_scores_actuals to public")

dfp.to_sql("pitchers_scores_ros", conn, schema="fantasy", if_exists="replace")
result = conn.execute("grant select on fantasy.pitchers_scores_ros to public")

dfpa.to_sql("pitchers_scores_actuals", conn, schema="fantasy", if_exists="replace")
result = conn.execute("grant select on fantasy.pitchers_scores_actuals to public")


## Create Draft Sheet

This sheet includes a number of important pieces of information for drafting specifically, so that players can look up by eligibility as well as important stats.

- Name
- Eligibility
- Positions
- Combined scores

In [ ]:
draft_sheet_query = """
select
    p.fullname,
    case when p.onteamid = 0 then 0 else 1 end as claimed,
    p.onteamid,
    p.eligibility,
    p.position as pos,
    p.injurystatus,
    upper(sc.catg) as catg,
    p.averagedraftposition as espn_adp,
    p.percentowned,
    p.percentstarted,
    sc.score
    -- sc_ros.score_ros
from fantasy.players p
    left join (
        select name, 'b' as catg, total_z_score as score
        from fantasy.batters_scores
        union all
        select name, 'p' as catg, total_z_score as score
        from fantasy.pitchers_scores
    ) sc
        on p.fullname = sc.name
        and case when p.eligibility not like '%UTIL%' then 'p' else 'b' end = sc.catg
    /* left join (
        select name, 'b' as catg, total_z_score as score_ros
        from fantasy.batters_scores_ros
        union all
        select name, 'p' as catg, total_z_score as score_ros
        from fantasy.pitchers_scores_ros
    ) sc_ros
        on p.fullname = sc_ros.name
        and case when p.eligibility not like '%UTIL%' then 'p' else 'b' end = sc_ros.catg */
"""
dfd = pd.read_sql(sqlalchemy.text(draft_sheet_query), conn)

In [ ]:
dfd.sort_values(by='score', inplace=True, ascending=False)
dfd['rank'] = dfd['score'].rank(ascending=False, method='first', na_option='bottom')
dfd.reset_index(drop=True)
print()

## Create Positional Scarcity Metrics

Players are grouped into the _most scarce_ position that they're eligible for. Scarcity is defined as number of players above replacement value. This creates a map of all positions and the number of players above replacement, and then uses that to group players to which they should be in.

In [ ]:
# create a map of positional scarcity
eligibility = {}
for index, player in dfd.iterrows():
    # add all positions to the dict if not yet
    for pos in player['eligibility'].split("|"):
        if pos not in eligibility:
            eligibility[pos] = 0

        if player['score'] >= 0.0:
            eligibility[pos] += 1

def get_scarcest_eligibility(x):
    """
    Loops through a player's eligibility and puts them in the category that has the least
    number of players that are above replacement level. Arbitrarily set max value unreachably
    high to start so that any eligibility validates.
    """
    best = ""
    best_num = 1000000000
    for pos in x.split("|"):
        pos_num = eligibility[pos]
        if pos_num < best_num:
            best = pos
            best_num = pos_num
    return best

# get the shallowest position that any player is eligible for, as well
# as their primary position (first in the list, generally)
dfd['shallowest_pos'] = dfd['eligibility'].apply(lambda x: get_scarcest_eligibility(x))
dfd['shallowest_pos_rank'] = dfd.groupby("shallowest_pos")["score"].rank("dense", ascending=False)
dfd['primary_pos'] = dfd['pos'].apply(lambda x: x.split("|")[0])
dfd['primary_pos_rank'] = dfd.groupby("primary_pos")["score"].rank("dense", ascending=False)
dfd['catg_rank'] = dfd.groupby("catg")["score"].rank("dense", ascending=False)

## Style Output

In order to make the outputs more usable, apply a number of color stylings and ranking scale sliders to the actual xlsx file.

In [ ]:
dfd['score_diff'] = dfd['score_ros'] - dfd['score']
dfd = dfd[['rank','fullname','onteamid', 'claimed','eligibility','pos','injurystatus','score','score_ros','score_diff','espn_adp','percentowned',
    'percentstarted','catg','catg_rank','shallowest_pos','shallowest_pos_rank','primary_pos','primary_pos_rank']]

## Write Excel files for distribution

In [ ]:
with pd.ExcelWriter('fantasy2019.xlsx') as writer:
    dfb.to_excel(writer, sheet_name='batters')
    dfp.to_excel(writer, sheet_name='pitchers')
    dfd.to_excel(writer, sheet_name='draft_sheet')

## Draft Sheet Preview

This is a quick look at how the players are going to appear in the final draft sheet. Useful for comparing overall pitcher v batter weights (i.e. verify that nobody should be ranked higher than trout).

In [ ]:
dfd[
    (dfd['percentowned'] > 5.0)
    & ((dfd['claimed'] == 0) | (dfd['onteamid'] == 5))
    & (dfd['catg'] == 'P')
].sort_values(by='score', ascending=False).head(150)


# Visualizations

Here are a number of helpful visualizations to sort out the information from the draft sheet.

### ESPN Average Draft Pos vs. this Draft Sheet Ranking

Here's how players stack up against how they're being drafted on ESPN right now

In [ ]:
sb.lmplot(x="score", y="espn_adp", hue="onteamid", fit_reg=False, data=dfd[
    (dfd['percentowned'] > 5.0) &
#     ((dfd['claimed'] == 0) | (dfd['onteamid'] == 5)) &
    (dfd['catg'] == 'P') &
    (dfd['injurystatus'] == 'ACTIVE')
].sort_values(by='score', ascending=False).head(150))


In [ ]:
sb.scatterplot(x="score", y="score_ros", hue="score_diff", size="score_diff",
          data=dfd[(dfd['percentowned'] > 20.0) & (dfd['claimed'] == 0) & (dfd['catg'] == 'B')])

Here are the biggest individual differences in ranking for the top 100 players by rank in this system. If the number is negative that means this system thinks that they're better than their espn adp.

In [ ]:
dfd[(dfd['percentowned'] > 20.0) & ((dfd['claimed'] == 0) | (dfd['onteamid'] == 5)) & (dfd['pos'] == 'RP')
   ].sort_values(by='score', ascending=False).head(100)

### Team Comparisons

The individual positions and their associated depth for this year.

In [ ]:
sb.barplot(x="catg", y="score", hue="onteamid", ci=None, data=dfd[dfd['onteamid'] > 0])